In [1]:
import pandas as pd
import pyarrow
import numpy as np
#import pmdarima

from sklearn.ensemble import RandomForestClassifier
from statsmodels.tsa.arima.model import ARIMA

import warnings

# Create dataframe for training and testing

In [24]:
dataframe = pd.read_parquet("/h/dsmith/physionet_data/prepped/physionet_data.parquet", engine="pyarrow")

In [25]:
dataframe

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,SepsisEver,id
0,60.0,98.0,36.200,135.0,109.0,72.0,17.0,33.017979,-0.441724,24.382587,...,194.00,57.0,0.0,0.492043,0.507957,-12.02,1.0,0.0,0,27338
1,60.0,98.0,36.200,135.0,109.0,72.0,17.0,33.017979,-0.441724,24.382587,...,194.00,57.0,0.0,0.492043,0.507957,-12.02,2.0,0.0,0,27338
2,56.0,98.0,36.225,127.0,105.0,73.0,15.0,33.017979,-0.441724,24.382587,...,194.00,57.0,0.0,0.492043,0.507957,-12.02,3.0,0.0,0,27338
3,54.0,98.0,36.250,119.0,100.0,76.0,13.0,33.017979,-0.441724,24.382587,...,194.00,57.0,0.0,0.492043,0.507957,-12.02,4.0,0.0,0,27338
4,52.0,98.0,36.275,119.0,87.0,67.0,12.0,33.017979,-0.441724,24.382587,...,194.00,57.0,0.0,0.492043,0.507957,-12.02,5.0,0.0,0,27338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429849,88.0,95.0,37.300,126.0,82.0,61.0,15.0,33.017979,0.800000,26.680000,...,81.96,76.7,0.0,0.000000,1.000000,-19.52,39.0,0.0,0,18000
1429850,88.0,98.0,37.400,118.0,75.0,57.0,20.0,33.017979,1.600000,26.760000,...,82.72,76.7,0.0,0.000000,1.000000,-19.52,40.0,0.0,0,18000
1429851,92.0,96.0,37.400,143.0,96.0,70.0,13.0,33.017979,2.400000,26.840000,...,83.48,76.7,0.0,0.000000,1.000000,-19.52,41.0,0.0,0,18000
1429852,95.0,96.0,37.600,129.0,84.0,61.0,15.5,33.017979,3.200000,26.920000,...,84.24,76.7,0.0,0.000000,1.000000,-19.52,42.0,0.0,0,18000


In [26]:
len(dataframe.id.unique())

36612

In [27]:
 # if static/non-static columns are not specified, then try to identify them by model itself

#print("dataframe.id: ", dataframe.id)
#print("dataframe.id[0]: ", dataframe.id[0])
df = dataframe.loc[dataframe.id == list(dataframe.id)[0]]
df = df.iloc[:,:-1]    
static_cols = [col for col in df.columns if np.var(df[col]) < 0.0000001]
if "ICULOS" in static_cols:
    static_cols.remove("ICULOS")

nonstatic_cols = list(set(dataframe.columns) - set(static_cols))
        

In [2]:
class ARIMAForest:
    def __init__(self, n_estimators=100, criterion='gini', max_depth=3,
                min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0,
                bootstrap=True, oob_score=False, n_jobs=2, random_state=None,
                verbose=0, warm_start=None, class_weight=None, ccp_alpha=0.0,
                max_samples=None, maxiter=50,
                 
                order=(3,0,3)):
        
        # inputs of random forest and arima models using sklearn and statsmodel
        self.n_estimators=n_estimators
        self.criterion=criterion 
        self.max_depth=max_depth
        self.min_samples_split=min_samples_split 
        self.min_samples_leaf=min_samples_leaf
        self.min_weight_fraction_leaf=min_weight_fraction_leaf
        self.max_features=max_features
        self.max_leaf_nodes=max_leaf_nodes
        self.min_impurity_decrease=min_impurity_decrease
        self.bootstrap=bootstrap
        self.oob_score=oob_score
        self.n_jobs=n_jobs
        self.random_state=random_state
        self.verbose=verbose
        self.warm_start=warm_start
        self.class_weight=class_weight
        self.ccp_alpha=ccp_alpha
        self.max_samples=max_samples
        self.maxiter = maxiter
        
        
        self.arima_order= order
        
        
    def ARIMA_Fit(self, dataframe, static_cols, nonstatic_cols):
        
        self.num_embedding = self.arima_order[0] + self.arima_order[2]
        
       
        #self.nonstatic_dataframe = dataframe[self.nonstatic_cols]
        
        
        #self.min_timeseries_stamps = min(dataframe.id.apply(lambda x: len(dataframe[dataframe.id == x])))
        #print(static_cols)
        
        # create an empty dataframe to store the results
        self.arima_results = pd.DataFrame(columns= [c + "_" + str(i) for c in nonstatic_cols for i in range(self.num_embedding)] + static_cols)
   
        i = 0
        for each_id in dataframe.id.unique():
            i += 1
            
            
            ###############################################################################################
            
            
            ##########                        Here， I only chose 20 instances to train the model.
            
            
            ################################################################################################
         
            if i == 21:
                break
                
                
                
                
            print(f"ID=={each_id}, {i}-th instance")
            
            if i%10 == 0:
                print(f"{i} out of {len(dataframe.id.unique())} is done.")
                
                
                
            #print(len(self.arima_results.columns))
            #print([0]*len(self.arima_results.columns))
            temp_predition_result = pd.DataFrame(data=[[0]*len(self.arima_results.columns)],
                                                 columns = self.arima_results.columns)
            
            
            df = dataframe.query("id=="+str(each_id))
            df = df.iloc[:10,:]
            
            if any(df.SepsisEver == 1):
                label = 1
            else:
                label = -1
            
            df.SepsisLabel = label
            
            for col in nonstatic_cols:
                y_train = df[col]
                forecaster = ARIMA(y_train, order=self.arima_order)
                
                forecaster_fit = forecaster.fit()
                
                
                embeddings = list(forecaster_fit.arparams) + list(forecaster_fit.maparams)
                
                #y_pred = list(forecaster.predict(fh=np.arange(self.future_horizon)+1))
            
                #print(y_pred)
                #print(temp_predition_result)
                for j in range(self.num_embedding):
                    #print(temp_predition_result.columns.get_loc(col + "_" + str(i)))
                    temp_predition_result.iloc[0, temp_predition_result.columns.get_loc(col + "_" + str(j))] = embeddings[j]
                    #print(temp_predition_result)
                    
            #print(temp_predition_result)
            for col in static_cols:
                temp_predition_result.iloc[0, temp_predition_result.columns.get_loc(col)] = df.iloc[0][col]
            #temp_predition_result.loc[self.static_cols] = df.iloc[0][self.static_cols]
            #print(temp_predition_result)
            
        #print(temp_predition_result)
        
            self.arima_results = self.arima_results.append(temp_predition_result)
        
        self.arima_results.reset_index(drop=True, inplace=True)

        return self.arima_results.loc[:, ~self.arima_results.columns.isin(['SepsisLabel','id'])], self.arima_results.SepsisLabel

            
            
    def RandomForest_Fit(self, X, y):
            
        self.clf = RandomForestClassifier(n_estimators=self.n_estimators, max_depth=self.max_depth, n_jobs=self.n_jobs) 
        
        """
         criterion=self.criterion, max_depth=self.max_depth,
                    min_samples_split=self.min_samples_split, min_samples_leaf=self.min_samples_leaf, min_weight_fraction_leaf=self.min_weight_fraction_leaf,
                    max_features=self.max_features, max_leaf_nodes=self.max_leaf_nodes, min_impurity_decrease=self.min_impurity_decrease,
                    bootstrap=self.bootstrap, oob_score=self.oob_score, n_jobs=self.n_jobs, random_state=self.random_state,
                    verbose=self.verbose, warm_start=self.warm_start, class_weight=self.class_weight, ccp_alpha=self.ccp_alpha,
                    max_samples=self.max_samples
        """
        
        self.clf.fit(X=X, y=y)
    
    
    def predict(self, X):
            
        return self.clf.predict(X)
    
    
        

In [3]:
arimaforest = ARIMAForest()

In [31]:
# warning messages tell us that:
# 1. 10 data points per time series is too short to confirm the trend. 
# 2. MLE estimate didn't converge.
warnings.filterwarnings("ignore")
id_train = dataframe.id.unique()[:len(dataframe.id.unique())//2]
id_test = dataframe.id.unique()[len(dataframe.id.unique())//2:]
dataframe_train = dataframe.loc[[True if id in id_train else False for id in dataframe.id ]]
dataframe_test = dataframe.loc[[True if id in id_test else False for id in dataframe.id ]]


In [42]:
X_train, y_train = arimaforest.ARIMA_Fit(dataframe=dataframe_train, static_cols=static_cols, nonstatic_cols=nonstatic_cols)

ID==27338, 1-th instance
ID==27339, 2-th instance
ID==27340, 3-th instance
ID==27341, 4-th instance
ID==27342, 5-th instance
ID==27343, 6-th instance
ID==27344, 7-th instance
ID==27345, 8-th instance
ID==27346, 9-th instance
ID==27347, 10-th instance
10 out of 18306 is done.
ID==27348, 11-th instance
ID==27349, 12-th instance
ID==27350, 13-th instance
ID==27351, 14-th instance
ID==27352, 15-th instance
ID==27353, 16-th instance
ID==27354, 17-th instance
ID==27355, 18-th instance
ID==27356, 19-th instance
ID==27357, 20-th instance
20 out of 18306 is done.


In [43]:
X_train.to_pickle("X_train.pkl")
y_train.to_pickle("y_train.pkl")

In [4]:
X_train = pd.read_pickle("X_train.pkl")
y_train = list(pd.read_pickle("y_train.pkl"))

In [5]:
arimaforest.RandomForest_Fit(X=X_train, y=y_train)

In [11]:
dataframe_test

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,SepsisEver,id
714927,74.0,100.0,36.100,142.0,93.0,66.0,16.0,26.000000,-0.441724,24.382587,...,55.615385,60.0,1.0,0.0,1.0,-0.10,21.0,0.0,0,28717
714928,84.0,99.0,36.200,152.0,96.0,72.0,18.0,24.000000,-0.441724,24.382587,...,55.153846,60.0,1.0,0.0,1.0,-0.10,22.0,0.0,0,28717
714929,66.0,100.0,36.300,159.0,111.0,76.0,18.0,24.000000,-0.441724,24.382587,...,54.692308,60.0,1.0,0.0,1.0,-0.10,23.0,0.0,0,28717
714930,70.0,100.0,36.275,188.0,127.0,88.0,16.0,24.000000,-0.441724,24.382587,...,54.230769,60.0,1.0,0.0,1.0,-0.10,24.0,0.0,0,28717
714931,72.0,100.0,36.250,162.0,101.0,87.0,15.0,24.000000,-0.441724,24.382587,...,53.769231,60.0,1.0,0.0,1.0,-0.10,25.0,0.0,0,28717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429849,88.0,95.0,37.300,126.0,82.0,61.0,15.0,33.017979,0.800000,26.680000,...,81.960000,76.7,0.0,0.0,1.0,-19.52,39.0,0.0,0,18000
1429850,88.0,98.0,37.400,118.0,75.0,57.0,20.0,33.017979,1.600000,26.760000,...,82.720000,76.7,0.0,0.0,1.0,-19.52,40.0,0.0,0,18000
1429851,92.0,96.0,37.400,143.0,96.0,70.0,13.0,33.017979,2.400000,26.840000,...,83.480000,76.7,0.0,0.0,1.0,-19.52,41.0,0.0,0,18000
1429852,95.0,96.0,37.600,129.0,84.0,61.0,15.5,33.017979,3.200000,26.920000,...,84.240000,76.7,0.0,0.0,1.0,-19.52,42.0,0.0,0,18000


In [46]:
X_test, y_test = arimaforest.ARIMA_Fit(dataframe=dataframe_test, static_cols=static_cols, nonstatic_cols=nonstatic_cols)


ID==28666, 1-th instance
ID==28667, 2-th instance
ID==28668, 3-th instance
ID==28669, 4-th instance
ID==28670, 5-th instance
ID==28671, 6-th instance
ID==28672, 7-th instance
ID==28673, 8-th instance
ID==28674, 9-th instance
ID==28675, 10-th instance
10 out of 18306 is done.
ID==28676, 11-th instance
ID==28677, 12-th instance
ID==28678, 13-th instance
ID==28679, 14-th instance
ID==28680, 15-th instance
ID==28681, 16-th instance
ID==28682, 17-th instance
ID==28683, 18-th instance
ID==28684, 19-th instance
ID==28685, 20-th instance
20 out of 18306 is done.


In [47]:
X_test.to_pickle("X_test.pkl")
y_test.to_pickle("y_test.pkl")


In [2]:
X_test = pd.read_pickle("X_test.pkl")
y_test = list(pd.read_pickle("y_test.pkl"))


In [7]:
prediction_results = arimaforest.predict(X_test)

In [9]:
prediction_results == y_test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [3]:
y_test

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1]